# 실시간 STT 변환 및 자동 대화 요약 서비스
동행매니저의 상담을 실시간으로 음성을 녹음한 후, 음성 데이터를 STT(Speech-to-Text)로 변환하고, 변환된 텍스트를 요약하는 서비스를 제공합니다. 

이 과정에서 비용효율적인 관리를 위해 텍스트 변환 이후, 더 이상 필요하지 않은 음성 파일들은 자동으로 삭제됩니다.

## 1. 필요한 라이브러리 임포트
- queue: 비동기적으로 녹음된 오디오 데이터를 관리합니다.
- threading: 녹음 작업을 백그라운드에서 실행하기 위해 스레드 처리를 담당합니다.
- sounddevice, soundfile: 오디오 녹음을 수행하고 WAV 파일로 저장하는 라이브러리입니다.
- pydub: WAV 파일을 20초 단위로 분할하는 데 사용됩니다.
- requests, json, base64, urllib3: HTTP 요청을 보내어 STT와 요약 API를 호출하기 위한 라이브러리입니다.
- os: 음성 파일을 삭제하기 위한 파일 시스템 작업을 수행하는 라이브러리입니다.

In [3]:
import queue
import threading
import sounddevice as sd
import soundfile as sf
import time
from pydub import AudioSegment
import requests
import json
import base64
import urllib3
import os  # 파일 삭제를 위해 os 모듈 추가


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## 2. 전역 변수 설정
- q: 녹음된 오디오 데이터를 저장하는 큐입니다.
- recording: 녹음 상태를 제어하는 플래그로, 녹음 중인지 여부를 나타냅니다.

In [4]:
q = queue.Queue()
recording = False

## 3. 음성 녹음 함수 정의

### 3.1 녹음된 오디오 데이터를 파일로 저장하는 함수

- complicated_record(): 마이크에서 입력된 오디오 데이터를 실시간으로 녹음하여 temp.wav 파일에 저장합니다.
- sf.SoundFile: WAV 파일을 생성합니다.
- sd.InputStream: 오디오 데이터를 스트리밍합니다.
- complicated_save(): 콜백 함수로, 데이터를 큐에 저장하여 파일에 쓰여지도록 합니다.

In [5]:
def complicated_record():
    with sf.SoundFile("temp.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())


### 3.2 녹음된 데이터를 큐에 저장하는 콜백 함수

- complicated_save(): 오디오 스트림에서 데이터를 받아 큐에 저장합니다.
- 이 함수는 sounddevice.InputStream에 의해 호출되며, 오디오 데이터를 큐에 복사합니다.

In [6]:
def complicated_save(indata, frames, time, status):
    q.put(indata.copy())

## 4. 녹음 시작 및 종료 함수 정의

### 4.1 녹음 시작 함수

- start_recording(): 녹음을 시작합니다.
- recording 플래그를 True로 설정하고, 녹음 작업을 별도의 스레드에서 실행하여 메인 프로그램이 중단되지 않도록 합니다.

In [7]:
def start_recording():
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('Start recording')
    recorder.start()
    return recorder

### 4.2 녹음 종료 함수
- stop_recording(): 녹음을 중지합니다.
- recording 플래그를 False로 설정하여 녹음 루프를 종료시키고, 스레드가 안전하게 종료될 때까지 기다립니다 (join() 사용).

In [8]:
def stop_recording(recorder):
    global recording
    recording = False
    recorder.join()
    print('Stop recording')


## 5. 녹음된 파일을 STT로 변환 및 파일 삭제

### 5.1 오디오 파일을 20초씩 분할하고 STT 처리

- split_audio_and_send(): 녹음된 파일을 20초 단위로 분할한 후, 각 분할된 파일을 STT로 변환하여 텍스트를 결합합니다.
- 분할된 파일 삭제: STT 변환이 완료된 후, 더 이상 필요하지 않은 분할된 WAV 파일을 삭제합니다.

In [11]:
def split_audio_and_send(file_path, segment_length=20000):
    audio = AudioSegment.from_wav(file_path)
    duration = len(audio)
    full_text = ""
    
    for i in range(0, duration, segment_length):
        segment = audio[i:i + segment_length]
        segment_path = f"segment_{i//1000}.wav"
        segment.export(segment_path, format="wav")
        text = send_to_stt(segment_path)
        full_text += text + " "
        
        # 변환된 후 분할된 음성 파일 삭제
        if os.path.exists(segment_path):
            os.remove(segment_path)

    return full_text.strip()


### 5.2 STT API 요청 함수

- send_to_stt(): 분할된 오디오 파일을 ETRI STT API로 전송하여 텍스트로 변환합니다.


In [13]:
def send_to_stt(segment_path):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
    accessKey = "ac4ce31e-ca09-4513-b6d9-a23ef73cc3ca"
    languageCode = "korean"

    with open(segment_path, "rb") as file:
        audioContents = base64.b64encode(file.read()).decode("utf8")
    
    requestJson = {
        "argument": {
            "language_code": languageCode,
            "audio": audioContents
        }
    }

    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization": accessKey},
        body=json.dumps(requestJson)
    )
    
    print(f"[responseCode] {response.status}")
    response_data = json.loads(response.data.decode('utf-8'))
    return response_data.get("return_object", {}).get("recognized", "")


### 5.3 요약 API 요청 함수

- summarize_text(): STT로 변환된 텍스트를 요약 API로 전송하여 요약된 텍스트를 반환받습니다.

In [15]:
def summarize_text(content):
    url = "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": "5pq84oiw69",
        "X-NCP-APIGW-API-KEY": "XqgnzxNQFrFJanNAeYE0DLZBZMgc6nt2pzv1PieZ",
        "Content-Type": "application/json"
    }

    data = {
        "document": {
            "title": "'음성 인식 결과 요약'",
            "content": content
        },
        "option": {
            "language": "ko",
            "model": "general",
            "tone": 3,
            "summaryCount": 5
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        summary = response.json().get("summary", "")
        print("요약된 텍스트:", summary)
    else:
        print(f"Error: {response.status_code}")
        print(response.json())


## 6. 전체 프로세스 실행

### 6.1 녹음 시작
- 60초 동안 녹음을 진행한 후, 녹음을 종료합니다.

In [9]:
recorder_thread = start_recording()
time.sleep(60)
stop_recording(recorder_thread)

Start recording
Stop recording


### 6.2 STT 처리 및 텍스트 요약
- 녹음된 파일을 STT로 변환한 후, 요약 API로 결과를 요약합니다.

In [16]:
full_text = split_audio_and_send("temp.wav")
print("Full Recognized Text:", full_text)

summarize_text(full_text)

[responseCode] 200
[responseCode] 200
[responseCode] 200
Full Recognized Text: 사랑은 물고 옮긴다 이 노래를 불러 산다는 나에게 마음 그들 하니 좋아 그대를 사랑하던 사람 그대 품에 그 사람 사랑해.
Error: 400
{'status': 400, 'error': {'errorCode': 'E100', 'message': 'Insufficient valid sentence'}}


### 6.3 원본 오디오 파일 삭제

STT 변환이 완료된 후, 원본 오디오 파일 temp.wav를 삭제하여 디스크 공간을 확보합니다.

In [17]:
# STT 변환 후 원본 음성 파일 삭제
if os.path.exists("temp.wav"):
    os.remove("temp.wav")
    print("Original audio file deleted.")

Original audio file deleted.
